In [107]:
#importing dependencies

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import sampler
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
from torchvision.utils import save_image

#import _pickle as cpickle #to store model histories in a file
import os
import imageio
from PIL import Image

use_cuda = False
device   = torch.device('cpu')
if torch.cuda.is_available():
    use_cuda = True
    device   = torch.device('cuda')
print(use_cuda)    
    
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots"

False


In [108]:
#specifying parameters
image_size = 64 #HACK to use MNIST architecture
G_input_dim = 100
G_output_dim = 3
D_input_dim = 3
D_output_dim = 1
num_filters = [1024, 512, 256, 128]

learning_rate = 0.0002
betas = (0.5, 0.999)
batch_size = 128
num_epochs = 1000

data_dir = './Train_data'
save_dir = './DCGAN_results/'

In [109]:
print(os.getcwd())

/Users/vivek1410patel/Documents/Quarter 3/CS231n/project/PoseGuided


In [110]:
#loading data

transform = transforms.Compose([
                                 #Hack to make MNIST code work
                                transforms.ToTensor(),
                                #transforms.Normalize(mean=(214.0466981, 206.55220904, 203.99178198), std=(54.34939265, 55.62690195, 58.85794001))
                               ])
                                

df_data = dsets.ImageFolder(data_dir, transform = transform)

data_loader = torch.utils.data.DataLoader(dataset=df_data,
                                          batch_size=batch_size,
                                          shuffle=True)


## VAE Model

In [111]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.fc1 = nn.Linear(64*64*3, 4096)
        self.fc2 = nn.Linear(4096,1000)
        self.fc3 = nn.Linear(1000,400)
        self.fc31 = nn.Linear(400, 60)
        self.fc32 = nn.Linear(400, 60)
        self.fc4 = nn.Linear(60, 400)
        self.fc5 = nn.Linear(400,1000)
        self.fc6 = nn.Linear(1000,4096)
        self.fc7 = nn.Linear(4096, 64*64*3)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        h2 = F.relu(self.fc2(h1))
        h3 = F.relu(self.fc3(h2))
        return self.fc31(h3), self.fc32(h3)

    def reparameterize(self, mu, logvar):
        if self.training:
            std = torch.exp(0.5*logvar)
            eps = torch.randn_like(std)
            return eps.mul(std).add_(mu)
        else:
            return mu

    def decode(self, z):
        h4 = F.relu(self.fc4(z))
        h5 = F.relu(self.fc5(h4))
        h6 = F.relu(self.fc6(h5))
        #out = F.sigmoid(self.fc7(h6))
        out = self.fc7(h6)
        return out

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 64*64*3))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

model = VAE().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

##  Loss Function

In [112]:
# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, mu, logvar):
    BCE = F.mse_loss(recon_x, x.view(-1, 64*64*3), size_average=False)

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return BCE + KLD

## Set up training and testing functions

In [113]:
def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(data_loader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model.forward(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(data_loader.dataset),
                100. * batch_idx / len(data_loader),
                loss.item() / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(data_loader.dataset)))


def test(epoch):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, (data, _) in enumerate(data_loader):
            data = data.to(device)
            recon_batch, mu, logvar = model(data)
            test_loss += loss_function(recon_batch, data, mu, logvar).item()
            if i == 0 and epoch%1 == 0:
                n = min(data.size(0), 8)
                comparison = torch.cat([data[:n],
                                      recon_batch.view(-1, 3, 64, 64)[:n]])
                save_image(comparison.cpu(),
                         'results/reconstruction_' + str(epoch) + '.png', nrow=n)

    test_loss /= len(data_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))


# Train and Test

In [ ]:
for epoch in range(1, num_epochs+1):
    train(epoch)
    test(epoch)
    
    with torch.no_grad():
        if epoch%1 == 0:
            sample = torch.randn(64, 60).to(device)
            sample = model.decode(sample).cpu()
            save_image(sample.view(64, 3, 64, 64),'results/sample_' + str(epoch) + '.png')


Train Epoch: 1 [0/32031 (0%)]	Loss: 8916.711914
Train Epoch: 1 [1280/32031 (4%)]	Loss: 1278.227173
Train Epoch: 1 [2560/32031 (8%)]	Loss: 863.892822
Train Epoch: 1 [3840/32031 (12%)]	Loss: 628.351868
Train Epoch: 1 [5120/32031 (16%)]	Loss: 570.920959
Train Epoch: 1 [6400/32031 (20%)]	Loss: 547.810303
Train Epoch: 1 [7680/32031 (24%)]	Loss: 509.180054
Train Epoch: 1 [8960/32031 (28%)]	Loss: 534.153076
Train Epoch: 1 [10240/32031 (32%)]	Loss: 474.596680
Train Epoch: 1 [11520/32031 (36%)]	Loss: 525.107727
Train Epoch: 1 [12800/32031 (40%)]	Loss: 516.339844
Train Epoch: 1 [14080/32031 (44%)]	Loss: 492.290985
Train Epoch: 1 [15360/32031 (48%)]	Loss: 506.296112
Train Epoch: 1 [16640/32031 (52%)]	Loss: 521.471375
Train Epoch: 1 [17920/32031 (56%)]	Loss: 490.244049
Train Epoch: 1 [19200/32031 (60%)]	Loss: 546.393250
Train Epoch: 1 [20480/32031 (64%)]	Loss: 475.197174
Train Epoch: 1 [21760/32031 (68%)]	Loss: 463.637238
Train Epoch: 1 [23040/32031 (72%)]	Loss: 488.050232
Train Epoch: 1 [24320/32

Train Epoch: 6 [29440/32031 (92%)]	Loss: 299.412781
Train Epoch: 6 [30720/32031 (96%)]	Loss: 313.778503
Train Epoch: 6 [7750/32031 (100%)]	Loss: 333.802514
====> Epoch: 6 Average loss: 313.8504
====> Test set loss: 319.5590
Train Epoch: 7 [0/32031 (0%)]	Loss: 320.965546
Train Epoch: 7 [1280/32031 (4%)]	Loss: 311.032776
Train Epoch: 7 [2560/32031 (8%)]	Loss: 297.501465
Train Epoch: 7 [3840/32031 (12%)]	Loss: 316.041016
Train Epoch: 7 [5120/32031 (16%)]	Loss: 316.380615
Train Epoch: 7 [6400/32031 (20%)]	Loss: 332.465271
Train Epoch: 7 [7680/32031 (24%)]	Loss: 317.655609
Train Epoch: 7 [8960/32031 (28%)]	Loss: 309.927399
Train Epoch: 7 [10240/32031 (32%)]	Loss: 297.116333
Train Epoch: 7 [11520/32031 (36%)]	Loss: 318.285034
Train Epoch: 7 [12800/32031 (40%)]	Loss: 307.377258
Train Epoch: 7 [14080/32031 (44%)]	Loss: 322.512970
Train Epoch: 7 [15360/32031 (48%)]	Loss: 288.918793
Train Epoch: 7 [16640/32031 (52%)]	Loss: 290.068054
Train Epoch: 7 [17920/32031 (56%)]	Loss: 324.738464
Train Epoc